**DANN MODEL**

In [0]:
import torch
import torch.nn as nn
from torch.utils.model_zoo import load_url as load_state_dict_from_url
from torch.autograd import Function

class ReverseLayerF(Function):

    # Forwards identity

    # Sends backward reversed gradients

    @staticmethod

    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod

    def backward(ctx, grad_output):
    #    print('backward')
        output = (grad_output.neg() * ctx.alpha)

        return output, None



model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class DANN(nn.Module):

    def __init__(self, num_classes=1000):
        super(DANN, self).__init__()
        # FEATURES EXTRACTOR

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

         # LABELS CLASSIFIER
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
         # DOMAIN CLASSIFIER
        self.domain_classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2),
        )



    def forward(self, x , alpha=None):
        x = self.features(x)
        features = self.avgpool(x)
        
        features = features.view(features.size(0), -1)
       # features = torch.flatten(x, 1)


        if alpha is not None:
           
            reverse_feature =  ReverseLayerF.apply(features, alpha)
            domain_out = self.domain_classifier(reverse_feature)
            return domain_out
        else:
            class_out = self.classifier(features)
            return class_out





def dann(pretrained=False, progress=True, **kwargs):

    model = DANN(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        
        model.load_state_dict(state_dict, strict=False )
        model.domain_classiﬁer[1].weight.data = model.classiﬁer[1].weight.data
        model.domain_classiﬁer[4].weight.data = model.classiﬁer[4].weight.data
        model.domain_classiﬁer[1].bias.data = model.classiﬁer[1].bias.data
        model.domain_classiﬁer[4].bias.data = model.classiﬁer[4].bias.data

    return model

In [0]:
PRINT_LOSS = False
DEVICE = 'cuda'

**FUNCTION**


In [0]:
# # By default, everything is loaded to cpu
# net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
def TrainModel(net,n_epoch,scheduler,source_dataloader,target_dataloader,criterion,optimizer):
  cudnn.benchmark # Calling this optimizes runtime
  current_step = 0
  # Start iterating over the epochs
  for epoch in range(n_epoch):
    #print('Starting epoch {}/{}, LR = {}'.format(epoch+1, n_epoch, scheduler.get_lr()))

    # Iterate over the dataset

    # len_dataloader = min(len(source_dataloader), len(target_dataloader))
    for source_images, labels in source_dataloader:
  
      # Bring data over the device of choice

    
      source_images = source_images.to(DEVICE)
      labels = labels.to(DEVICE)
      # target_images = target_images.to(DEVICE)
      # domain_label =  torch.zeros(len(source_images)).long().cuda()
      # target_domain_label =  torch.ones(len(target_images)).long().cuda()


      net.train() # Sets module in training mode
      #################################################
                      #TRAINING SOURCE CLASS
      #################################################
      # We need to manually set the gradients to zero before starting a new iteration
      optimizer.zero_grad() # Zero-ing the gradients
      outputs = net(source_images)
      
      loss = criterion(outputs, labels)
  
      # Log loss
      if(PRINT_LOSS == True):
        if current_step % LOG_FREQUENCY == 0:
          print('Step {}, Loss {}'.format(current_step, loss.item()))

      # Compute gradients for each layer and update weights
      loss.backward()  # backward pass: computes gradients

      optimizer.step() # update weights based on accumulated gradients

      current_step += 1
  
    # Step the scheduler
    scheduler.step() 


def TrainModelDANN(net,n_epoch,scheduler,source_dataloader,target_dataloader,criterion,optimizer,alpha):
  cudnn.benchmark # Calling this optimizes runtime
  current_step = 0
  L_t = []
  L_s = []
  L_p = []
  # Start iterating over the epochs
  for epoch in range(n_epoch):
    #print('Starting epoch {}/{}, LR = {}'.format(epoch+1, n_epoch, scheduler.get_lr()))

    # Iterate over the dataset

    len_dataloader = min(len(source_dataloader), len(target_dataloader))
    data_zip =zip(source_dataloader,target_dataloader)
    for (source_images, labels), (target_images, _) in data_zip:
  
      # Bring data over the device of choice


      # p = float(current_step + epoch * len_dataloader) / \
      #             NUM_EPOCHS / len_dataloader
      # alpha = 2. / (1. + np.exp(-10 * p)) - 1
      
    
      source_images = source_images.to(DEVICE)
      labels = labels.to(DEVICE)
      target_images = target_images.to(DEVICE)
      domain_label =  torch.zeros(len(source_images)).long().cuda()
      target_domain_label =  torch.ones(len(target_images)).long().cuda()


      net.train() # Sets module in training mode
      #################################################
                      #TRAINING SOURCE CLASS
      #################################################
      # We need to manually set the gradients to zero before starting a new iteration
      optimizer.zero_grad() # Zero-ing the gradients
      outputs = net(source_images,None)
      
      loss = criterion(outputs, labels)
  
      # Log loss
      if(PRINT_LOSS == True):
        if current_step % LOG_FREQUENCY == 0:
          print('Step {}, Loss {}'.format(current_step, loss.item()))
          L_p.append(loss)
          

      # Compute gradients for each layer and update weights
      loss.backward()  # backward pass: computes gradients


    #################################################
                      #TRAINING SOURCE DOMAIN
    #################################################
    # optimizer.zero_grad() # Zero-ing the gradients
      domain_outputs = net(source_images,alpha)


      loss = criterion(domain_outputs,domain_label)

      # Log loss
      if(PRINT_LOSS == True):
        if current_step % LOG_FREQUENCY == 0:
          print('Step {}, Loss {}'.format(current_step, loss.item()))
          L_s.append(loss)
      loss.backward() 
      
      #################################################
                      #TRAINING TARGET DOMAIN
      #################################################
  #   optimizer.zero_grad() # Zero-ing the gradients
      target_outputs = net(target_images,alpha)

      loss = criterion(target_outputs,target_domain_label)
      # Log loss
      if(PRINT_LOSS == True):
        if current_step % LOG_FREQUENCY == 0:
          print('Step {}, Loss {}'.format(current_step, loss.item()))
          L_t.append(loss)

      loss.backward() 

      optimizer.step() # update weights based on accumulated gradients

      current_step += 1
    # e.append(current_step)
    # L.append(loss)

    # Step the scheduler
    scheduler.step() 
  return L_s,L_t,L_p


# print('Test Accuracy: {}'.format(accuracy))
def testModel(model,testset): 
  # net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  model.train(False) # Set Network to evaluation mode

  running_corrects = 0
  for images, labels in testset:
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = model(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(testset.dataset))
#  print("Correctly predicted"running_corrects)
  print('Test Accuracy: {}'.format(accuracy))
  model.train(True)
  return float(accuracy)



**Install requirements**

In [0]:
# !pip3 install 'torch==1.3.1'
# !pip3 install 'torchvision==0.4.2'
# !pip3 install 'Pillow-SIMD'
# !pip3 install 'tqdm'

**Import libraries**

In [0]:
import os
import logging
import numpy as np
import torch

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

from PIL import Image
from tqdm import tqdm

**Set Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 7 # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 256    # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 0.01          # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 10     # Total number of training epochs (iterations over dataset)
STEP_SIZE =   200    # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

**Define Data Preprocessing**

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])                                   

**Prepare Dataset**

In [0]:
if not os.path.isdir('./Homework3-PACS'):
  !git clone https://github.com/MachineLearning2020/Homework3-PACS.git

DATA_DIR = 'Homework3-PACS/PACS'

art_dataset = torchvision.datasets.ImageFolder(DATA_DIR+"/art_painting", transform=train_transform)
photo_dataset = torchvision.datasets.ImageFolder(DATA_DIR+"/photo", transform=train_transform)
cartoon_dataset = torchvision.datasets.ImageFolder(DATA_DIR+"/cartoon", transform=train_transform)
sketch_dataset = torchvision.datasets.ImageFolder(DATA_DIR+"/sketch", transform=train_transform)


In [0]:
##Print Images
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

# import matplotlib as mpl
# mpl.rcParams['axes.grid'] = False
# mpl.rcParams['image.interpolation'] = 'nearest'
# mpl.rcParams['figure.figsize'] = 5, 5

# def show_dataset(dataset, n=1):
#   img = np.vstack((np.hstack((np.asarray(dataset[i][0]) for _ in range(n)))
#                    for i in range(len(dataset))))
#   plt.imshow(img.transpose(1, 2, 0))
#   plt.axis('off')

In [0]:

# try_dataset = Subset(photo_dataset, [1])
# show_dataset(try_dataset)
# try_dataset = Subset(art_dataset, [1])
# show_dataset(try_dataset)

**Prepare Dataloaders**

In [0]:

sketch_dataloader = DataLoader(sketch_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
cartoon_dataloader = DataLoader(cartoon_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
art_dataloader = DataLoader(art_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
photo_dataloader = DataLoader(photo_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

source_dataloader = photo_dataloader 
target_dataloader = art_dataloader

# len(train_dataloader.dataset)


**Prepare Network**

In [0]:

# net = dann(pretrained = True)
# net.classifier[6] = nn.Linear(4096, NUM_CLASSES)

**Prepare Training**

In [0]:


 
# parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

# domain_label = torch.zeros(BATCH_SIZE)

# domain_label = domain_label.long()
# domain_label = domain_label.cuda()


# alpha = 0.08

criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
DEVICE = 'cuda'
lr_list = [0.0001, 0.0005, 0.001,0.005,0.01]
alpha_list = [0.001,0.005,0.01,0.03,0.05,0.08,0.1]





**3.VALIDATION ON TEST SET WITHOUT DOMAIN ADAPTATION** 

In [0]:
# old_accuracy = 0
# for LR in lr_list:
#   net = alexnet(pretrained = True)
#   net.classifier[6] = nn.Linear(4096, NUM_CLASSES)
#   DEVICE = 'cuda'
#   net.to(DEVICE)
#   criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
#   parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
#   optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
#   scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
#   print("Hyperparameter Learning Rate = ", LR)  
#   #######################################
#   # TRAIN MODEL 
#   ########################################
#   TrainModel(net,10,scheduler,source_dataloader,target_dataloader,criterion,optimizer)
#   ########################################
#   # TEST MODEL
#   ########################################
#   accuracy = testModel(net,target_dataloader) # we test the model on the target_dataloader to do validation ("cheating")
#   print("Accuracy", accuracy)
#   if(accuracy > old_accuracy):
#     best_lr = LR
  
#     torch.save(net.state_dict(), 'model.pt')
#     old_accuracy = accuracy


**3.VALIDATION ON TEST SET WITH DOMAIN ADAPTATION** 



In [0]:
# NUM_EPOCHS = 5     # Total number of training epochs (iterations over dataset)
# old_accuracy = 0
# for LR in lr_list:
#   for alpha in alpha_list:
#     print("Hyperparameter alpha = ", alpha, "Learning Rate = ", LR)
#     net = dann(pretrained = True)
#     net.classifier[6] = nn.Linear(4096, NUM_CLASSES)
#     DEVICE = 'cuda'
#     net.to(DEVICE)
#     criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
#     parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
#     optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
#     scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
#     ########################################
#     # TRAIN MODEL 
#     ########################################
#     TrainModelDANN(net,NUM_EPOCHS,scheduler,source_dataloader,target_dataloader,criterion,optimizer,alpha)
#     ########################################
#     # TEST MODEL
#     ########################################
#     accuracy = testModel(net,target_dataloader) # we test the model on the target_dataloader to do validation ("cheating")
#     print("Accuracy", accuracy)
#     if(accuracy > old_accuracy):
#       best_lr = LR
#       best_alpha = alpha
#       torch.save(net.state_dict(), 'model.pt')
#       old_accuracy = accuracy


In [0]:
# print(best_alpha)
# print(best_lr)

In [0]:
# # By default, everything is loaded to cpu
# net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

# cudnn.benchmark # Calling this optimizes runtime
# # e = []
# # L = []

# current_step = 0
# # Start iterating over the epochs
# for epoch in range(NUM_EPOCHS):
#   print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

#   # Iterate over the dataset

#   len_dataloader = min(len(source_dataloader), len(target_dataloader))
#   data_zip =zip(source_dataloader,target_dataloader)
#   for (source_images, labels), (target_images, _) in data_zip:
 
#     # Bring data over the device of choice


#     # p = float(current_step + epoch * len_dataloader) / \
#     #             NUM_EPOCHS / len_dataloader
#     # alpha = 2. / (1. + np.exp(-10 * p)) - 1
   
#     source_images = source_images.to(DEVICE)
#     labels = labels.to(DEVICE)
#     target_images = target_images.to(DEVICE)
#     domain_label =  torch.zeros(len(source_images)).long().cuda()
#     target_domain_label =  torch.ones(len(target_images)).long().cuda()


#     net.train() # Sets module in training mode
#     #################################################
#                     #TRAINING SOURCE CLASS
#     #################################################
#     # We need to manually set the gradients to zero before starting a new iteration
#     optimizer.zero_grad() # Zero-ing the gradients
#     outputs = net(source_images,None)
    
#     loss = criterion(outputs, labels)
 
#     # Log loss
#     if current_step % LOG_FREQUENCY == 0:
#       print('Step {}, Loss {}'.format(current_step, loss.item()))

#     # Compute gradients for each layer and update weights
#     loss.backward()  # backward pass: computes gradients


#    #################################################
#                     #TRAINING SOURCE DOMAIN
#    #################################################
#    # optimizer.zero_grad() # Zero-ing the gradients
#     domain_outputs = net(source_images,alpha)


#     loss = criterion(domain_outputs,domain_label)

#     # Log loss
#     if current_step % LOG_FREQUENCY == 0:
#       print('Step {}, Loss {}'.format(current_step, loss.item()))
 
#     loss.backward() 
    
#     #################################################
#                     #TRAINING TARGET DOMAIN
#     #################################################
#  #   optimizer.zero_grad() # Zero-ing the gradients
#     target_outputs = net(target_images,alpha)

#     loss = criterion(target_outputs,target_domain_label)
#     # Log loss
#     if current_step % LOG_FREQUENCY == 0:
#       print('Step {}, Loss {}'.format(current_step, loss.item()))

#     loss.backward() 

#     optimizer.step() # update weights based on accumulated gradients

#     current_step += 1
#   # e.append(current_step)
#   # L.append(loss)

#   # Step the scheduler
#   scheduler.step() 

**4.CROSS DOMAIN VALIDATION**

**4.A VALIDATION WITHOUT ADAPTATION**



In [0]:
# NUM_EPOCHS = 10
# old_accuracy = 0
# for LR in lr_list:
#   net = alexnet(pretrained = True)
#   net.classifier[6] = nn.Linear(4096, NUM_CLASSES)
#   DEVICE = 'cuda'
#   net.to(DEVICE)
#   criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
#   parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
#   optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
#   scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
#   print("Hyperparameter Learning Rate = ", LR)  
#   #######################################
#   # TRAIN MODEL 
#   ########################################
#   TrainModel(net,NUM_EPOCHS,scheduler,source_dataloader,target_dataloader,criterion,optimizer)
#   ########################################
#   # TEST MODEL
#   ########################################
#   accuracy = testModel(net,sketch_dataloader) # we test the model on the target_dataloader to do validation ("cheating")
#   accuracy = accuracy + testModel(net,cartoon_dataloader)
#   accuracy = accuracy / 2
#   print("Accuracy", accuracy)
#   if(accuracy > old_accuracy):
#     best_lr = LR
#     torch.save(net.state_dict(), 'model.pt')
#     old_accuracy = accuracy

# print(best_lr)

**Test**


In [0]:

# net.load_state_dict(torch.load('model.pt'))
# testModel(net,art_dataloader)

**4.B VALIDATION WITH DOMAIN ADAPTATION**

In [0]:
source_dataloader = photo_dataloader 
target_dataloader = cartoon_dataloader
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
accuracy_list_cartoon = []
NUM_EPOCHS = 5     # Total number of training epochs (iterations over dataset)
old_accuracy = 0
for LR in lr_list:
  for alpha in alpha_list:
    print("Hyperparameter alpha = ", alpha, "Learning Rate = ", LR)
    net = dann(pretrained = True)
    net.classifier[6] = nn.Linear(4096, NUM_CLASSES)
    net = net.to(DEVICE)
    parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
    optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
    ########################################
    # TRAIN MODEL 
    ########################################
    TrainModelDANN(net,NUM_EPOCHS,scheduler,source_dataloader,target_dataloader,criterion,optimizer,alpha)
    ########################################
    # TEST MODEL
    ########################################
    accuracy = testModel(net,target_dataloader)
    print("accuracy", accuracy)
    accuracy_list_cartoon.append(accuracy)

In [0]:
source_dataloader = photo_dataloader 
target_dataloader = sketch_dataloader
accuracy_list_sketch = []
NUM_EPOCHS = 5     # Total number of training epochs (iterations over dataset)
old_accuracy = 0
for LR in lr_list:
  for alpha in alpha_list:
    print("Hyperparameter alpha = ", alpha, "Learning Rate = ", LR)
    net = dann(pretrained = True)
    net.classifier[6] = nn.Linear(4096, NUM_CLASSES)
    net.to(DEVICE)
    parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
    optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
    ########################################
    # TRAIN MODEL 
    ########################################
    TrainModelDANN(net,NUM_EPOCHS,scheduler,source_dataloader,target_dataloader,criterion,optimizer,alpha)
    ########################################
    # TEST MODEL
    ########################################
    accuracy = testModel(net,target_dataloader) # we test the model on the target_dataloader to do validation ("cheating")
    print("accuracy", accuracy) 
    accuracy_list_sketch.append(accuracy)

In [0]:
array_sketch = np.asarray(accuracy_list_sketch)
array_cartoon = np.asarray(accuracy_list_cartoon)
tot_accuracy_list = array_sketch + array_cartoon
index = np.argmax(tot_accuracy_list)
best_lr = lr_list[int(index/len(alpha_list))]
best_alpha = alpha_list[index%len(alpha_list)]
print(best_alpha)
print(best_lr)   

In [0]:



source_dataloader = photo_dataloader 
target_dataloader = art_dataloader
net = dann(pretrained = True)
net.classifier[6] = nn.Linear(4096, NUM_CLASSES)
net.to(DEVICE)
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
optimizer = optim.SGD(parameters_to_optimize, lr=best_lr, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
########################################
# TRAIN MODEL 
########################################
L_s,L_t,L_p = TrainModelDANN(net,10,scheduler,source_dataloader,target_dataloader,criterion,optimizer,best_alpha)
  

In [0]:

########################################
# TEST MODEL
########################################
print("art painting")
testModel(net,target_dataloader)
